In [109]:
import requests
import psycopg2
import polyline
import gpxpy
import json

import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx

In [110]:
OUTPUT_DIR = '../routes'

In [111]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [112]:
def get_route(points):
    params = {
        'geometries': 'polyline6',
    }
    response = requests.get(f"http://127.0.0.1:6000/route/v1/all/{points}", params=params)
    print(response.json())
    response.raise_for_status()
    route = response.json()
    return route['routes'][0]['geometry']

In [113]:
def get_nearest(point):
    response = requests.get(f"http://127.0.0.1:6000/nearest/v1/foot/{point}")
    response.raise_for_status()
    waypoints = response.json()
    location = waypoints['waypoints'][0]['location']
    return f"{location[0]},{location[1]}"

In [114]:
def plot_gpx(gpx_df, filename):
    plt.figure(figsize=(15, 8))
    plt.plot(gpx_df['longitude'], gpx_df['latitude'], color='red', label='Route')
    ctx.add_basemap(plt.gca(), crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
    plt.legend()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)
    plt.savefig(f"{OUTPUT_DIR}/{filename}.png")

In [115]:
query = """
SELECT
    b.name AS destination,
    a.latitude AS start_lat,
    a.longitude AS start_lon, 
    b.latitude AS stop_lat,
    b.longitude AS stop_lon
FROM start a
CROSS JOIN destinations b
"""

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()

df = pd.DataFrame(rows, columns=[
    "destination",
    "start_lat", 
    "start_lon", 
    "stop_lat",
    "stop_lon",
])

df.head(3)

,destination,start_lat,start_lon,stop_lat,stop_lon
0,Merlion,1.3096438,103.8966102,1.2869149,103.8547688
1,Marina Bay Financial Center,1.3096438,103.8966102,1.2821578,103.854498
2,Meadow,1.3096438,103.8966102,1.2795255,103.8622062


In [116]:
for index, row in df.iterrows():    
    start_location = get_nearest(f"{row['start_lon']},{row['start_lat']}")
    end_location = get_nearest(f"{row['stop_lon']},{row['stop_lat']}")
    
    points=f"{start_location};{end_location}"
    print(points)
    
    try:
        geometry = get_route(points)
        coordinates = polyline.decode(geometry, 6)
    except:
        continue

#     gpx_df = pd.DataFrame(coordinates, columns=['latitude', 'longitude'])
#     plot_gpx(gpx_df, row['destination'])

#     distance = round(row['distance_to_destination'] / 1000.0)
#     filename = f"{row['destination']} ({distance}km)"
#     create_gpx(coordinates, filename)

103.89661,1.30963;103.854589,1.286915
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.854498,1.282148
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.861956,1.279454
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.847621,1.291848
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.870546,1.280678
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.861596,1.286978
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.899685,1.296921
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.870873,1.316492
{'message': 'Impossible route between points', 'code': 'NoRoute'}
103.89661,1.30963;103.859081,1.269395
{'message': 'Impossible route between points', 'code': 'NoRoute'}
